# 価格は適正

今日は、パズルのもう 1 つのピース、つまり RSS フィードを購読して有望な取引を探す ScanningAgent を構築します。

（OpenAI の構造化出力（Structured Output）機能。以前はこういうコードは書けなかったが、LLMで実現可能になってきた。）

In [1]:
# import

import os
import json
from dotenv import load_dotenv
from openai import OpenAI

# `./agents/deals.py`
from agents.deals import ScrapedDeal, DealSelection

# `./agents/scanner_agent.py`
from agents.scanner_agent import ScannerAgent

In [2]:
# 初期化と定数

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

## ScrapedDeal, DealSelection

In [3]:
# 複数のRSSフィードからScrapedDealのリストを生成
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:14<00:00, 14.81s/it]


In [4]:
# 最新5件 * 5 RSSフィード = 25件
len(deals)

25

In [5]:
# 0-4 が Electronics、5-9 が Computers、10-14 が Automotive、15-19 が Smart-Home、20-24 が Home-Garden、
deals[22].describe()

"Title: Harbor Freight Tools Scratch and Dent Sale: Up to 45% off + free shipping\nDetails: Save up to 45% off a range of tool storage chests and stainless steel worktops. We've pictured the Icon 19 In. Professional End Cabinet for $629 ($270 off) Stock is limited. Items ship for free, although you're responsible for un-palletizing and setup. You need to call the number on the sale page to place an order for these items. Shop Now at Harbor Freight Tools\nFeatures: \nURL: https://www.dealnews.com/Harbor-Freight-Tools-Scratch-and-Dent-Sale-Up-to-45-off-free-shipping/21760435.html?iref=rss-c196"

In [6]:
# OpenAI の構造化出力（Structured Output）機能を使うプロンプト
# JSON手書きかよ...と思ったが、JSONは必須ではなくっなっているらしい。

system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

_system_prompt = """リストから、最も詳細で質の高い説明と明確な価格を持つ5つの取引を抽出し、要約します。
この形式を使用し、説明を一切含まないJSON形式で応答してください。価格は説明から導き出された数値で提供してください。取引の価格が明確でない場合は、その取引を応答に含めないでください。
最も重要なのは、価格を含む最も詳細な商品説明を持つ5つの取引を応答することです。取引条件に言及することは重要ではなく、商品の詳細な説明です。
「$XXXオフ」や「$XXX値下げ」と説明されている商品には注意してください。これは商品の実際の価格ではありません。価格に十分な確信がある場合にのみ、商品情報を応答してください。

{"deals": [
    {
        "product_description": "4～5文で商品の概要を明確に表現してください。なぜお得なのかよりも、商品の詳細の方がはるかに重要です。割引やクーポンに関する言及は避け、商品そのものに焦点を当ててください。選択した商品ごとに段落ごとにテキストを記載する必要があります。",
        "price": 99.99,
        "url": "提供されたURL"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""

_user_prompt = """このリストから、最も有望な5つの取引を返信してください。中でも、最も詳細で質の高い商品説明と明確な価格があるものを選んでください。
返信はJSON形式のみで行ってください。商品説明は、取引条件ではなく、商品自体の概要となるように書き換えてください。
選択した5つの商品それぞれについて、product_descriptionフィールドに1段落分のテキストを返信してください。
「$XXXオフ」や「$XXX値下げ」などと記載されている商品にはご注意ください。これは実際の価格ではありません。価格に十分な確信がある場合のみ、商品情報を返信してください。

取引：

"""

user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
# 先頭2000文字表示
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: EcoFlow Delta 2 1,024Wh Portable Power Station for $490 + free shipping
Details: It's $19 under Amazon and the lowest price we could find. Buy Now at Walmart
Features: AC charge input and output  solar input (solar panel not included)  4 USB charging ports 2 USB-C charging ports Model: EFD330-2
URL: https://www.dealnews.com/products/Eco-Flow/Eco-Flow-Delta-2-1-024-W

In [9]:
def get_recommendations():
    # OpenAI の構造化出力（Structured Output）機能
    completion = openai.beta.chat.completions.parse(
        # OpenAIのモデル
        model="gpt-4o-mini",
        # モデルへのプロンプト
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        # Python の特定の型（DealSelection）...
        response_format=DealSelection
    )
    # 実行結果をDealSelection型にパース
    result = completion.choices[0].message.parsed
    return result # DealSelection = Dealのlist

In [10]:
# 構造化出力
result = get_recommendations()
print(len(result.deals))
print(result)

5
deals=[Deal(product_description='The EcoFlow Delta 2 is a portable power station featuring a 1,024Wh capacity, making it perfect for camping trips or emergency power supply. With multiple charging options, including AC, USB, and solar inputs, it ensures that your devices stay powered even off grid. It includes four USB charging ports and two USB-C ports for versatile charging needs. The model is identified as EFD330-2, emphasizing its robust design and reliability for outdoor activities and home backups.', price=490.0, url='https://www.dealnews.com/products/Eco-Flow/Eco-Flow-Delta-2-1-024-Wh-Portable-Power-Station/473744.html?iref=rss-c142'), Deal(product_description="The Open-Box Beats Pill Portable Bluetooth Speaker delivers high-quality sound with a maximum output of 100W. This speaker supports Bluetooth and USB connections, ensuring versatile compatibility with most devices. It's rated IP67, providing excellent protection against dust and water, making it ideal for outdoor use. A

## ScannerAgent
上記を全てラップしたエージェントを実行

In [11]:
# RSSフィードをスキャンし構造化出力
agent = ScannerAgent()
result = agent.scan()
print(result)

deals=[Deal(product_description="The EcoFlow Delta 2 is a powerful portable power station with a capacity of 1,024Wh, perfect for outdoor adventures or emergency backup. It features an AC charge input and output, allowing you to power devices efficiently. With four USB charging ports and two USB-C charging ports, this power station offers versatility for various electronic devices. Whether it's charging phones or running small appliances, the Delta 2 ensures you stay connected and powered up even off the grid.", price=490.0, url='https://www.dealnews.com/products/Eco-Flow/Eco-Flow-Delta-2-1-024-Wh-Portable-Power-Station/473744.html?iref=rss-c142'), Deal(product_description='The Beats Pill Portable Bluetooth Speaker is a compact yet powerful audio device with a maximum output of 100W. It supports Bluetooth and USB connections and is designed to be IP67-rated, making it both dust and water-resistant. This speaker can pair with other Beats Pill speakers for an enhanced sound experience, w